# Training

In [ ]:
import core.preprocessing as pp
from core.neural_net_forecaster import NeuralNetForecaster

# Train on the large, but test on the small
dm = pp.DataManager()

dm.load_all_and_split(kind='all', reindex=True)
dftrain = dm.detrended_train

dm.load_all_and_split(kind='small', reindex=True)
dfdev = dm.detrended_dev

# RUN PARAMETERS
batchsize = 500
niter = 6 * 150000 // batchsize 

problem = NeuralNetForecaster(dftrain, dfdev, arch='FC1', logdir='./tmp/debug/FC1-detrend',
                              niter=niter, batchsize=batchsize,
                              learningrate=0.01, sampling="rand")

problem.make_forecasts()

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


# Prediction

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk')

problem.plot_test()

ax = plt.gca()
lim = ax.get_xlim()
chunk = (lim[1] - lim[0])/8.
start = lim[0] + 2*chunk
end = lim[0] + 6*chunk

ax.set_xlim(start, end)

plt.show()

In [ ]:
# Add the forecasts to the DataManager class
dm.add_forecasts(problem.forecasts)
# Use data manager class to change all indices back to "real time"
dm.swap_index()

In [ ]:
# With the forecasts in "real time" the default retrending function may be used
transformed_forecasts = [pp.retrend_data(f) for f in dm.forecasts]

In [ ]:
# Translate back to "sequential time" for plotting
dm.add_forecasts(transformed_forecasts)
dm.swap_index()

In [ ]:
from core.utilities import plot_forecasts

plot_forecasts(dm.original_dev, dm.forecasts)
ax = plt.gca()
lim = ax.get_xlim()
chunk = (lim[1] - lim[0])/8.
start = lim[0] + 2*chunk
end = lim[0] + 6*chunk
ax.set_ylim(-50, 350)
ax.set_xlim(start, end)

In [ ]:
problem.calc_mse()